In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image

In [15]:
def load_and_preprocess_image(img_path, target_size, preprocess_input_fn):
    img = image.load_img(img_path, target_size=target_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input_fn(x)
    return x

img_path = 'elefante2.webp'

In [16]:
# ResNet50

from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions

model_resnet50 = ResNet50(weights='imagenet')
img_resnet = load_and_preprocess_image(img_path, (224, 224), preprocess_input)
preds_resnet = model_resnet50.predict(img_resnet)

print("ResNet50 Prediction:")
print(decode_predictions(preds_resnet, top=1)[0][0])
print()

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
ResNet50 Prediction:
('n04399382', 'teddy', np.float32(0.80986404))



In [17]:
# InceptionV3

from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input as inception_preprocess, decode_predictions as inception_decode

model_inception = InceptionV3(weights='imagenet')
img_inception = load_and_preprocess_image(img_path, (299, 299), inception_preprocess)
preds_inception = model_inception.predict(img_inception)

print('InceptionV3 Predictions:')
print(inception_decode(preds_inception, top=1)[0][0])
print()

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
InceptionV3 Predictions:
('n04399382', 'teddy', np.float32(0.9672541))



In [18]:
# MobileNetV2

from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input as mobilenet_preprocess, decode_predictions as mobilenet_decode

model_mobilenet = MobileNetV2(weights='imagenet')
img_mobilenet = load_and_preprocess_image(img_path, (224, 224), mobilenet_preprocess)
preds_mobilenet = model_mobilenet.predict(img_mobilenet)

print('MobileNetV2 Predictions:')
print(mobilenet_decode(preds_mobilenet, top=1)[0][0])
print()

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 763ms/step
MobileNetV2 Predictions:
('n04399382', 'teddy', np.float32(0.9858964))



In [19]:
# DenseNet121

from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input as densenet_preprocess, decode_predictions as densenet_decode

model_densenet = DenseNet121(weights='imagenet')
img_densenet = load_and_preprocess_image(img_path, (224, 224), densenet_preprocess)
preds_densenet = model_densenet.predict(img_densenet)

print('DenseNet121 Predictions:')
print(densenet_decode(preds_densenet, top=1)[0][0])
print()

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
DenseNet121 Predictions:
('n04399382', 'teddy', np.float32(0.9774095))



In [20]:
# EfficientNetB0

from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocess, decode_predictions as efficientnet_decode

model_efficientnet = EfficientNetB0(weights='imagenet')
img_efficientnet = load_and_preprocess_image(img_path, (224, 224), efficientnet_preprocess)
preds_efficientnet = model_efficientnet.predict(img_efficientnet)

print('EfficientNetB0 Predictions:')
print(efficientnet_decode(preds_efficientnet, top=1)[0][0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
EfficientNetB0 Predictions:
('n04399382', 'teddy', np.float32(0.5606458))


In [19]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar100
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = cifar100.load_data()

y_train = tf.keras.utils.to_categorical(y_train, 100)
y_test = tf.keras.utils.to_categorical(y_test, 100)

def preprocess(image, label):
    image = tf.image.resize(image, (128, 128))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

for layer in base_model.layers:
    layer.trainable = False 

for layer in base_model.layers[-20:]:  
    layer.trainable = True 

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
 
predictions = Dense(100, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#model.summary()

model.fit(train_ds, epochs=10, validation_data=test_ds)

Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 227s 142ms/step - accuracy: 0.3103 - loss: 2.8550 - val_accuracy: 0.2845 - val_loss: 4.3732
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 225s 144ms/step - accuracy: 0.5091 - loss: 1.8296 - val_accuracy: 0.4446 - val_loss: 3.2883
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 221s 142ms/step - accuracy: 0.5804 - loss: 1.5202 - val_accuracy: 0.4816 - val_loss: 2.4845
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 222s 142ms/step - accuracy: 0.6289 - loss: 1.2972 - val_accuracy: 0.4932 - val_loss: 2.7047
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 221s 142ms/step - accuracy: 0.6767 - loss: 1.1115 - val_accuracy: 0.5076 - val_loss: 2.5553
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 223s 143ms/step - accuracy: 0.7191 - loss: 0.9518 - val_accuracy: 0.4963 - val_loss: 3.3625
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 229s 146ms/step - accuracy: 0.7534 - loss: 0.8154 - val_accuracy: 0.5507 - val_loss: 2.7102
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 232s 148ms/step - ac